In [1]:

import librosa
import soundfile
import os, glob
import numpy as np
import pickle, joblib


# Modules to load data and extract features

In [6]:
# Extract features : mfcc, chroma, mel

def extract_feature(file, mfcc, chroma, mel):
    with soundfile.SoundFile(file) as file:
        X = file.read(dtype="float32")
        sample_rate=file.samplerate
        
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        
        return result

In [8]:
# Emotions in the RAVDESS dataset

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Emotions to observe
observed_emotions = ['neutral','happy','angry','sad']
# observed_emotions = ['neutral','calm','happy','sad','angry','fearful','disgust','surprised']

In [9]:

# Load the data and extract features for each sound file

wav_files = 'C:\\Users\\admin\\Downloads\\bdad-project\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav'
# wav_files = 'C:\\Users\\admin\\Downloads\\bdad-project\\sample-ravdess\\Actor_*\\*.wav'

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(wav_files):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


# Creating and training the ML model

In [20]:
# creating the model

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = load_data(test_size=0.25)

model= MLPClassifier(
            alpha=0.01, 
            batch_size=10, 
            epsilon=1e-08, 
            hidden_layer_sizes=(300,300), 
            learning_rate='adaptive', 
            max_iter=500,
            verbose=True )

model.fit(x_train, y_train)

print('test score: ')
model.score(x_test, y_test)


Iteration 1, loss = 8.41624275
Iteration 2, loss = 3.16654037
Iteration 3, loss = 3.52444921
Iteration 4, loss = 2.76568082
Iteration 5, loss = 2.66886089
Iteration 6, loss = 2.61508936
Iteration 7, loss = 1.64033360
Iteration 8, loss = 2.51695730
Iteration 9, loss = 2.09422028
Iteration 10, loss = 1.64861419
Iteration 11, loss = 2.62241075
Iteration 12, loss = 1.68293334
Iteration 13, loss = 1.54929481
Iteration 14, loss = 1.63129548
Iteration 15, loss = 1.59621575
Iteration 16, loss = 1.20009299
Iteration 17, loss = 1.14247698
Iteration 18, loss = 0.91748540
Iteration 19, loss = 1.35753978
Iteration 20, loss = 1.01725971
Iteration 21, loss = 1.01902313
Iteration 22, loss = 0.88974705
Iteration 23, loss = 0.99254995
Iteration 24, loss = 1.01937519
Iteration 25, loss = 0.75927876
Iteration 26, loss = 1.69065352
Iteration 27, loss = 0.98474146
Iteration 28, loss = 1.00695165
Iteration 29, loss = 0.76215256
Iteration 30, loss = 0.81348290
Iteration 31, loss = 0.60603783
Iteration 32, los

0.7083333333333334

In [21]:
model.score(x_train, y_train)

0.9722222222222222

In [16]:
model.get_params()

{'activation': 'relu',
 'alpha': 0.01,
 'batch_size': 10,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (300,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [15]:
model.predict_proba(x_test[:1])
# model.predict(x_test[:1])

array([[1.24517921e-02, 1.32700245e-01, 8.54841013e-01, 6.95050077e-06]])

# Serializing the model

In [25]:
# serialising the model object so we dont have to retrain it every time we run jupyter nb

with open('E:\\Speech Emotion Analysis\\python-ml-notebook\\models\\model_fulldata_all_51_84', 'wb') as file:
    pickle.dump(model, file)
    # pickle.dump(model, model_pickle.pkl)


In [16]:

# file names are in format: model_<fulldata/partial data>_<numberOfEmotions>_TestDataAccuracy_TrainDataAccuracy

models = [
    'E:\\Speech Emotion Analysis\\python-ml-notebook\\models\\model_fulldata_4_60_78',
    'E:\\Speech Emotion Analysis\\python-ml-notebook\\models\\model_fulldata_4_63_73',
    'E:\\Speech Emotion Analysis\\python-ml-notebook\\models\\model_fulldata_all_51_84'
]

with open(models[1], 'rb') as file:
    model_unpickled = pickle.load(file)

model_unpickled.score(x_test, y_test)


0.6309523809523809

In [9]:
# recording sound and storing as a wave file
# REDUNDANT MODULE: voice recording functionality has been implemented in the frontend

# import sounddevice 
# from scipy.io.wavfile import write

# fs = 44100
# seconds = 5
# print("recording...")
# record_voice = sounddevice.rec(int(seconds*fs), samplerate=fs, channels=2)
# sounddevice.wait()
# write("C:\\Users\\admin\\Downloads\\bdad-project\\rec-voice.wav", fs, record_voice)
# print("DONE")


recording...
DONE


# Using the model to predict emotion of recorded voice sample

In [24]:
# searching mp3 files in Downloads folder

mp3_files = glob.glob("C:\\Users\\admin\\Downloads\\*.mp3")
print(mp3_files)


['C:\\Users\\admin\\Downloads\\56551fa4-eba0-4712-9252-d93cfec103c4.mp3', 'C:\\Users\\admin\\Downloads\\a178cf81-fb6e-4448-919d-a0a91230877e.mp3']


In [25]:
# Converting mp3 files to .wav format

for file in mp3_files:
    # convert wav to mp3 using ffmpeg
    # ffmpeg is a seperate tool and runs on the command line
    os.system(f"""ffmpeg -i {file} -acodec pcm_u8 -ar 22050 {file[:-4]}.wav""")   

print('Converted to wave')

Converted to wave


In [26]:
# Extracting features from recorded voice

file = 0
wav_files = glob.glob('C:\\Users\\admin\\Downloads\\*.wav')

for wav_file in wav_files:
    file = wav_file

features = extract_feature(file, mfcc=True, chroma=True, mel=True)


In [27]:
# Predicting the emotion

model.predict([features])

array(['disgust'], dtype='<U9')